In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pywt
from math import log10, sqrt
import pydicom as dcm
import os
import pandas as pd
import csv
import imutils

def img_read(x):
    img=cv2.imread(x,0)
    return img
def watermark_read(x):
    wtr_img=cv2.imread(x,0)
    wtr_img=cv2.resize(wtr_img,(64,64))
    return wtr_img

import random
def hcf(a,b):
    if(b%a==0):
        return a
    else:
        return hcf(b%a,a)
def randomseq(size):
    random.seed(size)
    
    num=[x for x in range(size)]
    h=len(num)
    for m in range(h):
        if m>2:
            r=hcf(m,h)
            if r==1:
                break 
    i=random.randint(0,size-1)    
    x=i
    res=[]
    while True:
        i=(i+m)%h
        if i==x:
            break
        res.append(i)
        
    res.append(x)
    return res

def toBinary(j):
    temp=bin(j).replace('0b',"")
    temp2=temp[::-1]
    while len(temp2)<8:
        temp2+='0'
    temp=temp2[::-1]
    if(j<0):
        temp=temp.replace('-',"")
        temp[1:]
        temp="1"+temp
    return temp
def toDecimal(j):
    temp=str(j)
    temp2=temp[::-1]
    while(len(temp2)<8):
        temp2+='0'
    temp=temp2[::-1]
    j=temp
    # if(j[0]=="1"):
    #     j=j[1:]
    #     return -int(j,2)
    # else:
    #     j=j[1:]
    #     return int(j,2)
    return int(j,2)
def binarycoversion(wtr_img):
    list_wtr_img=""
    for i in wtr_img:
        for j in i:
            list_wtr_img+=toBinary(j)
    return list_wtr_img

def embedding(img,list_wtr_img):
    # wrow,wcol=wtr_img.shape
    (row,col)=img.shape
    # LL,(HL,LH,HH)=pywt.dwt2(img,wavelet='haar')
    LL=img
    lrow,lcol=LL.shape
    l4x4=LL.reshape(-1,2,2)
    ustore=[]
    vstore=[]
    sstore=[]
    t=0
    print("2x2 Blocks:")
    print(l4x4)
    for i in l4x4:
        u,s,v=np.linalg.svd(i)
        t+=abs(u[0][0]-u[1][0])
        # u[1,0],u[2,0]=u[2,0],u[1,0]
        ustore.append(u)
        vstore.append(v)
        sstore.append(s)
    t=t/len(ustore)
    # print(f"Value of t is {t}")
    newu=[]
    ccc=0
    for i in ustore:
        newu.append(i)
        print(f"U matrix {ccc}")
        ccc+=1
        print(i)
    # print(ustore)
    # print(list_wtr_img)
    # len(ustore)
    seq=randomseq(len(ustore))
    k=0
    print(f"Sequence is {seq}")
    for i in seq:
        if k==len(list_wtr_img):
            break
        if list_wtr_img[k]=='0':
            if ustore[i][0][0]<=ustore[i][1][0]:
                ustore[i][0][0],ustore[i][1][0]=ustore[i][1][0],ustore[i][0][0]
            if ustore[i][0][0]-ustore[i][1][0]<t:
                ustore[i][0][0]+=t/2
                ustore[i][1][0]-=t/2
        else:
            if ustore[i][0][0]>ustore[i][1][0]:
                ustore[i][0][0],ustore[i][1][0]=ustore[i][1][0],ustore[i][0][0]
            if ustore[i][1][0]-ustore[i][0][0]<t:
                ustore[i][1][0]+=t/2
                ustore[i][0][0]-=t/2
        k+=1
    # print(ustore)
    updatedu=[]
    ccc=0
    for i in ustore:
        updatedu.append(i)
        print(f"Updated U {ccc}")
        ccc+=1
        print(i)
    newll=[]
    for i in range(len(sstore)):
        u=ustore[i]
        v=vstore[i]
        s=sstore[i]
        temp=u@np.diag(s)@v
        newll.append(temp)
        print(f"Updated block {i+1}")
        print(temp.astype(int))
    newll=np.array(newll)
    newll=newll.ravel()
    newll=newll.reshape(lrow,lcol)
    # plt.imshow(newll,cmap='gray')
    # watermarked_img=pywt.idwt2((newll,(HL,LH,HH)),wavelet='haar')
    return newu,updatedu,newll

def recover(watermarked_img,l):
    # dLL,(dHL,dLH,dHH)=pywt.dwt2(watermarked_img,wavelet='haar')
    dLL=watermarked_img
    dl2x2=dLL.reshape(-1,2,2)
    rec=''

    seq=randomseq(dl2x2.shape[0])
    k=0
    for i in seq:
        if k==l:
            break
        u,s,v=np.linalg.svd(dl2x2[i])
        print(i)
        print(u)
        if u[0][0]>u[1][0]:
            rec+='0'
        else:
            rec+='1'
        k+=1
    # generated_wtr=np.zeros((wrow,wcol),dtype='uint8')
    # count=0
    # for i in range(wrow):
    #     for j in range(wcol):
    #         temp=rec[count:count+8]
    #         count+=8
    #         generated_wtr[i,j]=toDecimal(temp)
    # return generated_wtr
    return rec

In [17]:
image=img_read("./Coverimages/airplane.tiff")
test=image[0:8,0:8]
# watermark=watermark_read("./Coverimages/airplane.tiff")
# twtr=watermark[0:1,0:1]
twtr="10111000"

In [18]:
u,upu,stego=embedding(test,twtr)

2x2 Blocks:
[[[ 65 199]
  [193 185]]

 [[179 185]
  [191 189]]

 [[ 54 196]
  [192 195]]

 [[195 192]
  [191 192]]

 [[ 54 197]
  [194 197]]

 [[195 191]
  [191 188]]

 [[ 55 187]
  [187 188]]

 [[186 180]
  [180 182]]

 [[ 62 181]
  [180 180]]

 [[180 183]
  [182 183]]

 [[ 69 174]
  [174 178]]

 [[179 179]
  [181 186]]

 [[ 72 180]
  [179 174]]

 [[174 181]
  [179 180]]

 [[ 77 176]
  [171 168]]

 [[170 175]
  [169 174]]]
U matrix 0
[[-0.60428147 -0.79677093]
 [-0.79677093  0.60428147]]
U matrix 1
[[-0.69178059 -0.72210776]
 [-0.72210776  0.69178059]]
U matrix 2
[[-0.58034118 -0.81437345]
 [-0.81437345  0.58034118]]
U matrix 3
[[-0.71078004 -0.70341434]
 [-0.70341434  0.71078004]]
U matrix 4
[[-0.57782807 -0.81615851]
 [-0.81615851  0.57782807]]
U matrix 5
[[-0.71355502 -0.7005992 ]
 [-0.7005992   0.71355502]]
U matrix 6
[[-0.57660239 -0.8170249 ]
 [-0.8170249   0.57660239]]
U matrix 7
[[-0.71102406 -0.70316768]
 [-0.70316768  0.71102406]]
U matrix 8
[[-0.58874325 -0.8083201 ]
 [-0.8

In [19]:
rec=recover(stego,8)

14
[[-0.79057967 -0.6123592 ]
 [-0.6123592   0.79057967]]
1
[[-0.63281132 -0.77430603]
 [-0.77430603  0.63281132]]
4
[[-0.8276538  -0.56123897]
 [-0.56123897  0.8276538 ]]
7
[[-0.76422731 -0.64494699]
 [-0.64494699  0.76422731]]
10
[[-0.81228151 -0.58326558]
 [-0.58326558  0.81228151]]
13
[[-0.644986   -0.76419438]
 [-0.76419438  0.644986  ]]
0
[[-0.60428147 -0.79677093]
 [-0.79677093  0.60428147]]
3
[[-0.6452157  -0.76400046]
 [-0.76400046  0.6452157 ]]


In [20]:
u=np.array(u)
upu=np.array(upu)
u.shape

(16, 2, 2)